![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)




[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/tutorials/streamlit_notebooks/public/QUESTION_ANSWERING_OPEN_BOOK.ipynb)

# **QUESTION ANSWERING (Open Book Questions)**




# **Colab Setup and Start Spark Session**

In [ ]:
!pip install -q pyspark==3.3.0 spark-nlp==4.0.2

In [2]:
import sparknlp
import pandas as pd

spark = sparknlp.start()


from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType, IntegerType



print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark 


Spark NLP version 4.0.2
Apache Spark version: 3.3.0


# **Question Answering with `T5_small` and `T5_base` models**

In [3]:
model_list = ['t5_base', 't5_small']

In [42]:
question_context_list = [
    ["""What does increased oxygen concentrations in the patient’s lungs displace?""" """context : Hyperbaric (high-pressure) medicine uses special oxygen chambers to increase the partial pressure of O 2 around the patient and, when needed, the medical staff. Carbon monoxide poisoning, gas gangrene, and decompression sickness (the ’bends’) are sometimes treated using these devices. Increased O 2 concentration in the lungs helps to displace carbon monoxide from the heme group of hemoglobin. Oxygen gas is poisonous to the anaerobic bacteria that cause gas gangrene, so increasing its partial pressure helps kill them. Decompression sickness occurs in divers who decompress too quickly after a dive, resulting in bubbles of inert gas, mostly nitrogen and helium, forming in their blood. Increasing the pressure of O 2 as soon as possible is part of the treatment."""],
    ["""What category of game is Legend of Zelda: Twilight Princess?""" """context: The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed and published by Nintendo for the GameCube and Wii home video game consoles. It is the thirteenth installment in the The Legend of Zelda series. Originally planned for release on the GameCube in November 2005, Twilight Princess was delayed by Nintendo to allow its developers to refine the game, add more content, and port it to the Wii. The Wii version was released alongside the console in North America in November 2006, and in Japan, Europe, and Australia the following month. The GameCube version was released worldwide in December 2006."""],
    ["""When did Beyonce start becoming popular?""" """context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy"."""],
    ["""For what instrument did Frédéric write primarily for?""" """context: Frédéric François Chopin (/ˈʃoʊpæn/; French pronunciation: ​[fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃]; 22 February or 1 March 1810 – 17 October 1849), born Fryderyk Franciszek Chopin,[n 1] was a Polish and French (by citizenship and birth of father) composer and a virtuoso pianist of the Romantic era, who wrote primarily for the solo piano. He gained and has maintained renown worldwide as one of the leading musicians of his era, whose "poetic genius was based on a professional technique that was without equal in his generation." Chopin was born in what was then the Duchy of Warsaw, and grew up in Warsaw, which after 1815 became part of Congress Poland. A child prodigy, he completed his musical education and composed his earlier works in Warsaw before leaving Poland at the age of 20, less than a month before the outbreak of the November 1830 Uprising."""],
    ["""The most populated city in the United States is which city?""" """context: New York—often called New York City or the City of New York to distinguish it from the State of New York, of which it is a part—is the most populous city in the United States and the center of the New York metropolitan area, the premier gateway for legal immigration to the United States and one of the most populous urban agglomerations in the world. A global power city, New York exerts a significant impact upon commerce, finance, media, art, fashion, research, technology, education, and entertainment, its fast pace defining the term New York minute. Home to the headquarters of the United Nations, New York is an important center for international diplomacy and has been described as the cultural and financial capital of the world."""],
]

In [46]:
for i in model_list:
  document_assembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("documents")

  t5 = T5Transformer() \
      .pretrained("t5_base") \
      .setTask("question:")\
      .setMaxOutputLength(200)\
      .setInputCols(["documents"]) \
      .setOutputCol("answers")

  t5_pp = Pipeline(stages=[
      document_assembler, 
      t5])

  df = spark.createDataFrame(question_context_list).toDF('text')
  model = t5_pp.fit(df).transform(df)
  print(f"MODEL NAME : {i}")
  model.select(['text','answers.result']).show(truncate=50)


t5_base download started this may take some time.
Approximate size to download 451.8 MB
[OK!]
MODEL NAME : t5_base
+--------------------------------------------------+------------------+
|                                              text|            result|
+--------------------------------------------------+------------------+
|What does increased oxygen concentrations in th...| [carbon monoxide]|
|What category of game is Legend of Zelda: Twili...|[action-adventure]|
|When did Beyonce start becoming popular?context...|           [False]|
|For what instrument did Frédéric write primaril...|      [solo piano]|
|The most populated city in the United States is...|        [New York]|
+--------------------------------------------------+------------------+

t5_base download started this may take some time.
Approximate size to download 451.8 MB
[OK!]
MODEL NAME : t5_small
+--------------------------------------------------+------------------+
|                                              t

## **`bert_qa_callmenicky_finetuned_squa` models**

In [22]:
sample_texts = [
    ["When was the first nutrition experiment performed?", """Sometimes overlooked during his life, James Lind, a physician in the British navy, performed the first scientific nutrition experiment in 1747. Lind discovered that lime juice saved sailors that had been at sea for years from scurvy, a deadly and painful bleeding disorder. Between 1500 and 1800, an estimated two million sailors had died of scurvy. The discovery was ignored for forty years, after which British sailors became known as "limeys." The essential vitamin C within citrus fruits would not be identified by scientists until 1932."""],
    ["What is regarded as the greatest literary work in Old English?", """The first example is taken from the opening lines of the folk-epic Beowulf, a poem of some 3,000 lines and the single greatest work of Old English. This passage describes how Hrothgar's legendary ancestor Scyld was found as a baby, washed ashore, and adopted by a noble family. The translation is literal and represents the original poetic word order. As such, it is not typical of Old English prose. The modern cognates of original words have been used whenever practical to give a close approximation of the feel of the original poem."""],
    ["What is the social style of hunter-gather societies?", """Hunter-gatherers tend to have an egalitarian social ethos, although settled hunter-gatherers (for example, those inhabiting the Northwest Coast of North America) are an exception to this rule. Nearly all African hunter-gatherers are egalitarian, with women roughly as influential and powerful as men."""],
    ["Through what part of the body are nutrients transported to feed cells?", """Animal tissue consists of elements and compounds ingested, digested, absorbed, and circulated through the bloodstream to feed the cells of the body. Except in the unborn fetus, the digestive system is the first system involved[vague]. Digestive juices break chemical bonds in ingested molecules, and modify their conformations and energy states. Though some molecules are absorbed into the bloodstream unchanged, digestive processes release them from the matrix of foods. Unabsorbed matter, along with some waste products of metabolism, is eliminated from the body in the feces."""],
    ["What type of flight decks are aircraft carriers equipped with?", """An aircraft carrier is a warship that serves as a seagoing airbase, equipped with a full-length flight deck and facilities for carrying, arming, deploying, and recovering aircraft."""]    
]


df = spark.createDataFrame(sample_texts).toDF("question", "context")


In [4]:
documentAssembler = MultiDocumentAssembler() \
        .setInputCols(["question", "context"]) \
        .setOutputCols(["document_question", "document_context"])


spanClassifier = BertForQuestionAnswering.pretrained("bert_qa_callmenicky_finetuned_squad","en") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer")\
    .setMaxSentenceLength(50)\
    .setCaseSensitive(False)

pipeline = Pipeline(stages=[documentAssembler, 
                            spanClassifier])

pipeline_model = pipeline.fit(spark.createDataFrame([['']]).toDF('text'))

bert_qa_callmenicky_finetuned_squad download started this may take some time.
Approximate size to download 385.6 MB
[OK!]


In [23]:
model = pipeline.fit(df).transform(df)

In [24]:
model.select("question", "answer.result").show(truncate=False)


+----------------------------------------------------------------------+---------------+
|question                                                              |result         |
+----------------------------------------------------------------------+---------------+
|When was the first nutrition experiment performed?                    |[1747]         |
|What is regarded as the greatest literary work in Old English?        |[Beowulf]      |
|What is the social style of hunter-gather societies?                  |[egalitarian]  |
|Through what part of the body are nutrients transported to feed cells?|[bloodstream]  |
|What type of flight decks are aircraft carriers equipped with?        |[full - length]|
+----------------------------------------------------------------------+---------------+



# **`roberta_qa_CV_Merge_DS`model**

In [30]:
spanClassifier = RoBertaForQuestionAnswering.pretrained("roberta_qa_CV_Merge_DS", "en") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer") \
    .setCaseSensitive(True)

pipeline = Pipeline().setStages([
    documentAssembler,
    spanClassifier
    ])

pipeline_model = pipeline.fit(spark.createDataFrame([[""]]).toDF('text'))

roberta_qa_CV_Merge_DS download started this may take some time.
Approximate size to download 442.6 MB
[OK!]


In [46]:
sample_texts = [
    ["When was the first nutrition experiment performed?", """Sometimes overlooked during his life, James Lind, a physician in the British navy, performed the first scientific nutrition experiment in 1747. Lind discovered that lime juice saved sailors that had been at sea for years from scurvy, a deadly and painful bleeding disorder. Between 1500 and 1800, an estimated two million sailors had died of scurvy. The discovery was ignored for forty years, after which British sailors became known as "limeys." The essential vitamin C within citrus fruits would not be identified by scientists until 1932."""],
    ["What is regarded as the greatest literary work in Old English?", """The first example is taken from the opening lines of the folk-epic Beowulf, a poem of some 3,000 lines and the single greatest work of Old English. This passage describes how Hrothgar's legendary ancestor Scyld was found as a baby, washed ashore, and adopted by a noble family. The translation is literal and represents the original poetic word order. As such, it is not typical of Old English prose. The modern cognates of original words have been used whenever practical to give a close approximation of the feel of the original poem."""],
    ["What is the social style of hunter-gather societies?", """Hunter-gatherers tend to have an egalitarian social ethos, although settled hunter-gatherers (for example, those inhabiting the Northwest Coast of North America) are an exception to this rule. Nearly all African hunter-gatherers are egalitarian, with women roughly as influential and powerful as men."""],
    ["Through what part of the body are nutrients transported to feed cells?", """Animal tissue consists of elements and compounds ingested, digested, absorbed, and circulated through the bloodstream to feed the cells of the body. Except in the unborn fetus, the digestive system is the first system involved[vague]. Digestive juices break chemical bonds in ingested molecules, and modify their conformations and energy states. Though some molecules are absorbed into the bloodstream unchanged, digestive processes release them from the matrix of foods. Unabsorbed matter, along with some waste products of metabolism, is eliminated from the body in the feces."""],
    ["What type of flight decks are aircraft carriers equipped with?", """An aircraft carrier is a warship that serves as a seagoing airbase, equipped with a full-length flight deck and facilities for carrying, arming, deploying, and recovering aircraft."""]    
]

In [47]:
df = spark.createDataFrame(sample_texts).toDF("question", "context")
model = pipeline.fit(df).transform(df)

In [48]:
model.select("question", "answer.result").show(truncate=False)

+----------------------------------------------------------------------+------------------+
|question                                                              |result            |
+----------------------------------------------------------------------+------------------+
|When was the first nutrition experiment performed?                    |[1747]            |
|What is regarded as the greatest literary work in Old English?        |[folk-epicBeowulf]|
|What is the social style of hunter-gather societies?                  |[egalitarian]     |
|Through what part of the body are nutrients transported to feed cells?|[]                |
|What type of flight decks are aircraft carriers equipped with?        |[full-length]     |
+----------------------------------------------------------------------+------------------+



# **`alert_qa_xxlarge_tweetqa` model**

In [35]:
spanClassifier = AlbertForQuestionAnswering.pretrained("albert_qa_xxlarge_tweetqa","en") \
  .setInputCols(["document_question", "document_context"]) \
  .setOutputCol("answer")

pipeline = Pipeline(stages=[
    documentAssembler, 
    spanClassifier
    ])

pipeline_model = pipeline.fit(spark.createDataFrame([['']]).toDF('text'))

albert_qa_xxlarge_tweetqa download started this may take some time.
Approximate size to download 735.8 MB
[OK!]


In [63]:
sample_texts = [
    ["When was the first nutrition experiment performed?", """Sometimes overlooked during his life, James Lind, a physician in the British navy, performed the first scientific nutrition experiment in 1747. Lind discovered that lime juice saved sailors that had been at sea for years from scurvy, a deadly and painful bleeding disorder. Between 1500 and 1800, an estimated two million sailors had died of scurvy. The discovery was ignored for forty years, after which British sailors became known as "limeys." The essential vitamin C within citrus fruits would not be identified by scientists until 1932."""],
    ["What is the social style of hunter-gather societies?", """Hunter-gatherers tend to have an egalitarian social ethos, although settled hunter-gatherers (for example, those inhabiting the Northwest Coast of North America) are an exception to this rule. Nearly all African hunter-gatherers are egalitarian, with women roughly as influential and powerful as men."""],
    ["Who is responsible for defence and foreign affairs?", """Defence and foreign affairs are carried out by the United Kingdom, which also retains responsibility to ensure good government. It must approve any changes to the Constitution of Bermuda."""],
    ["What is my name?", "My name is Clara and I live in Berkeley."],
    ["What type of flight decks are aircraft carriers equipped with?", """An aircraft carrier is a warship that serves as a seagoing airbase, equipped with a full-length flight deck and facilities for carrying, arming, deploying, and recovering aircraft."""],
]

In [64]:
df = spark.createDataFrame(sample_texts).toDF("question", "context")

model = pipeline.fit(df).transform(df)

In [65]:
model.select("question", "answer.result").show(truncate=False)

+--------------------------------------------------------------+------------------+
|question                                                      |result            |
+--------------------------------------------------------------+------------------+
|When was the first nutrition experiment performed?            |[1747]            |
|What is the social style of hunter-gather societies?          |[egalitarian]     |
|Who is responsible for defence and foreign affairs?           |[theUnitedKingdom]|
|What is my name?                                              |[Clara]           |
|What type of flight decks are aircraft carriers equipped with?|[full-length]     |
+--------------------------------------------------------------+------------------+

